In [0]:
from pyspark.sql import SparkSession,functions as func

In [0]:
from pyspark.sql.types import StructType,StructField,StringType,FloatType,LongType,IntegerType,DoubleType

In [0]:
spark=SparkSession.builder.appName("Cosine Similarity").getOrCreate()

In [0]:
schema1=StructType([StructField("movieId",IntegerType(),True),StructField("title",StringType(),True),StructField("genres",StringType(),True)])

In [0]:
movies=spark.read.schema(schema1).option("header","true").csv("dbfs:/FileStore/tables/movies.csv")
movies.show(5)

+-------+--------------------+--------------------+
movieId| title| genres|
+-------+--------------------+--------------------+
 1| Toy Story (1995)|Adventure|Animati...|
 2| Jumanji (1995)|Adventure|Childre...|
 3|Grumpier Old Men ...| Comedy|Romance|
 4|Waiting to Exhale...|Comedy|Drama|Romance|
 5|Father of the Bri...| Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows

In [0]:
movies.printSchema()

root
-- movieId: integer (nullable = true)
-- title: string (nullable = true)
-- genres: string (nullable = true)

In [0]:
schema2=StructType([StructField("userId",IntegerType(),True),StructField("movieId",IntegerType(),True),StructField("rating",DoubleType(),True),StructField("timestamp",StringType(),True)])

In [0]:
ratings=spark.read.schema(schema2).option("header","true").csv("dbfs:/FileStore/tables/ratings.csv")
ratings.show(5)

+------+-------+------+----------+
userId|movieId|rating| timestamp|
+------+-------+------+----------+
 1| 296| 5.0|1147880044|
 1| 306| 3.5|1147868817|
 1| 307| 5.0|1147868828|
 1| 665| 5.0|1147878820|
 1| 899| 3.5|1147868510|
+------+-------+------+----------+
only showing top 5 rows

In [0]:
ratings.printSchema()

root
-- userId: integer (nullable = true)
-- movieId: integer (nullable = true)
-- rating: double (nullable = true)
-- timestamp: string (nullable = true)

In [0]:
rating=ratings.select("userId","movieId","rating")
rating.show(5)

+------+-------+------+
userId|movieId|rating|
+------+-------+------+
 1| 296| 5.0|
 1| 306| 3.5|
 1| 307| 5.0|
 1| 665| 5.0|
 1| 899| 3.5|
+------+-------+------+
only showing top 5 rows

In [0]:
moviePairs=rating.alias("ratings1").join(rating.alias("ratings2"),\
           (func.col("ratings1.userId")==func.col("ratings2.userId")) & (func.col("ratings1.movieId")<func.col("ratings2.movieId"))) \
           .select(func.col("ratings1.movieId").alias("movie1"), \
           func.col("ratings2.movieId").alias("movie2"), \
           func.col("ratings1.rating").alias("rating1"), \
           func.col("ratings2.rating").alias("rating2"))

In [0]:
moviePairs.show(10)

+------+------+-------+-------+
movie1|movie2|rating1|rating2|
+------+------+-------+-------+
 10| 11| 3.5| 3.5|
 10| 17| 3.5| 2.5|
 10| 32| 3.5| 2.0|
 10| 34| 3.5| 3.5|
 10| 36| 3.5| 2.5|
 10| 39| 3.5| 2.0|
 10| 47| 3.5| 3.0|
 10| 50| 3.5| 3.0|
 10| 95| 3.5| 2.0|
 10| 110| 3.5| 1.5|
+------+------+-------+-------+
only showing top 10 rows

In [0]:
def computeCosineSimilarity(spark,df):
  pairScores=df.withColumn("xx",func.col("rating1")*func.col("rating1")) \
             .withColumn("yy",func.col("rating2")*func.col("rating2")) \
             .withColumn("xy",func.col("rating1")*func.col("rating2"))
  
  calculateSimilarity=pairScores.groupBy("movie1","movie2").agg(func.sum(func.col("xy")).alias("numerator"), \
                      (func.sqrt(func.sum(func.col("xx"))) * func.sqrt(func.sum(func.col("yy")))).alias("denominator"), \
                      func.count(func.col("xy")).alias("numPairs"))
  
  result=calculateSimilarity.withColumn("score",func.when(func.col("denominator")!=0,func.col("numerator")/func.col("denominator")).otherwise(0)) \
        .select("movie1","movie2","score","numPairs")
  return result

In [0]:
moviePairsSimilarity=computeCosineSimilarity(spark,moviePairs).cache()

In [0]:
moviePairsSimilarity.show(10)

+------+------+------------------+--------+
movie1|movie2| score|numPairs|
+------+------+------------------+--------+
 260| 912|0.9659099058784798| 740|
 541| 1196|0.9718928890210474| 1125|
 608| 1250|0.9600904313964878| 362|
 608| 3000|0.9647834127783648| 266|
 904| 1172|0.9722384468685841| 156|
 908| 919|0.9727964028869734| 341|
 912|134130| 0.962212592585605| 113|
 919| 926|0.9645907678951438| 109|
 926| 1217|0.9734740856019662| 38|
 926| 2502| 0.974770715647386| 40|
+------+------+------------------+--------+
only showing top 10 rows

In [0]:
scoreThreshold=0.95
coOccurrenceThreshold=50.0
movie_id=926

In [0]:
filtered_results=moviePairsSimilarity.filter(((func.col("movie1")==movie_id) | (func.col("movie2")==movie_id)) & \
                 (func.col("score") > scoreThreshold) & (func.col("numPairs")>coOccurrenceThreshold))

In [0]:
filtered_results.show(10)

+------+------+------------------+--------+
movie1|movie2| score|numPairs|
+------+------+------------------+--------+
 919| 926|0.9645907678951438| 109|
 926| 953|0.9668525580831191| 84|
 25| 926|0.9549608445715286| 54|
 926| 1674|0.9790753442948046| 52|
 926| 1961|0.9576583113415748| 76|
 926| 1172|0.9678841611555097| 62|
 926| 1086|0.9878582153074223| 58|
 926| 1199|0.9701664875702418| 65|
 926| 4963|0.9660358597193633| 55|
 926| 1228|0.9826296270863719| 76|
+------+------+------------------+--------+
only showing top 10 rows

In [0]:
filtered_results.sort("score",ascending=False).show(10)

+------+------+------------------+--------+
movie1|movie2| score|numPairs|
+------+------+------------------+--------+
 926| 1086|0.9878582153074223| 58|
 926| 3435|0.9871131974455927| 72|
 926| 930|0.9869985957471461| 63|
 909| 926|0.9859670875572815| 73|
 926| 1252| 0.983866287235212| 103|
 926| 1945| 0.983629262293348| 69|
 926| 1212|0.9835718382378524| 79|
 913| 926|0.9829377093793944| 96|
 926| 1228|0.9826296270863719| 76|
 905| 926|0.9821645551642226| 63|
+------+------+------------------+--------+
only showing top 10 rows

In [0]:
result=filtered_results.sort(func.col("score").desc()).take(10)

In [0]:
def getMovieName(movieNames,movie_id):
    result = movieNames.filter(func.col("movieId") == movie_id).select("title").collect()[0]
    return result[0]

In [0]:
print ("Top 10 similar movies for " + getMovieName(movies,movie_id))
for r in result:
  similarMovieID = r.movie1
  if(similarMovieID == movie_id):
    similarMovieID = r.movie2
  print(getMovieName(movies,similarMovieID) + "\tscore: " + str(r.score) + "\tstrength: " + str(r.numPairs))

Top 10 similar movies for All About Eve (1950)
Dial M for Murder (1954)	score: 0.9878582153074223	strength: 58
Double Indemnity (1944)	score: 0.9871131974455927	strength: 72
Notorious (1946)	score: 0.9869985957471461	strength: 63
Apartment, The (1960)	score: 0.9859670875572815	strength: 73
Chinatown (1974)	score: 0.983866287235212	strength: 103
On the Waterfront (1954)	score: 0.983629262293348	strength: 69
Third Man, The (1949)	score: 0.9835718382378524	strength: 79
Maltese Falcon, The (1941)	score: 0.9829377093793944	strength: 96
Raging Bull (1980)	score: 0.9826296270863719	strength: 76
It Happened One Night (1934)	score: 0.9821645551642226	strength: 63